# 다중회귀를 이용한 네번째 모델

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

train = pd.read_csv('/content/drive/MyDrive/영화 관객 수 예측/movies_train.csv')
test = pd.read_csv('/content/drive/MyDrive/영화 관객 수 예측/movies_test.csv')
submission = pd.read_csv('/content/drive/MyDrive/영화 관객 수 예측/submission.csv')

In [3]:
train.loc[train['dir_prev_num']==0] # 이전 작품 수가 0이면 평균 관객 수가 Nan값인 것을 알 수 있음. => 0으로 대체

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num
0,개들의 전쟁,롯데엔터테인먼트,액션,2012-11-22,96,청소년 관람불가,조병옥,NaN,0,91,2,23398
6,길위에서,백두대간,다큐멘터리,2013-05-23,104,전체 관람가,이창재,NaN,0,32,5,53526
8,"1789, 바스티유의 연인들",유니버설픽쳐스인터내셔널코리아,뮤지컬,2014-09-18,129,전체 관람가,정성복,NaN,0,3,5,4778
9,청춘그루브,(주)두타연,드라마,2012-03-15,94,15세 관람가,변성현,NaN,0,138,3,868
10,AV 아이돌,(주) 케이알씨지,멜로/로맨스,2015-07-27,89,청소년 관람불가,조조 히데오,NaN,0,0,4,745
...,...,...,...,...,...,...,...,...,...,...,...,...
593,낭만파 남편의 편지,퍼스트런,미스터리,2013-09-12,97,청소년 관람불가,최위안,NaN,0,5,3,1132
594,오빠가 돌아왔다,(주)마운틴픽쳐스,코미디,2014-03-20,94,청소년 관람불가,노진수,NaN,0,37,5,1958
597,살인의 강,(주)마운틴픽쳐스,공포,2010-09-30,99,청소년 관람불가,김대현,NaN,0,123,4,2475
598,악의 연대기,CJ 엔터테인먼트,느와르,2015-05-14,102,15세 관람가,백운학,NaN,0,431,4,2192525


In [4]:
train.fillna(0, inplace=True)
test.fillna(0, inplace=True)

In [5]:
train.drop(['title', 'distributor', 'director', 'release_time'], axis=1, inplace=True)
test.drop(['title', 'distributor', 'director', 'release_time'], axis=1, inplace=True)

In [6]:
# 장르와 상영시간을 레이블 인코딩
from sklearn.preprocessing import LabelEncoder
for col in ['genre', 'screening_rat']:
  encoder = LabelEncoder()
  train[col] = encoder.fit_transform(train[col])
  test[col] = encoder.transform(test[col]) # train데이터의 범주로 훈련시킨 그대로 test에 적용

In [7]:
# 학습데이터와 정답데이터 구분 
features = ['genre', 'time', 'screening_rat', 'dir_prev_bfnum',	'dir_prev_num',	'num_staff', 'num_actor']
target = train[['box_off_num']]
train = train[features]

In [8]:
# 피쳐 엔지니어링
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures()
poly.fit(train)
train_poly = poly.transform(train)
print(train_poly.shape)

(600, 36)


In [9]:
test_poly = poly.transform(test)
print(test_poly.shape)

(243, 36)


In [10]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_poly)
train_scaled = ss.transform(train_poly)
test_scaled = ss.transform(test_poly)

## 모델학습

In [11]:
from sklearn.linear_model import Ridge
rd = Ridge()
rd.fit(train_scaled, target)
print(rd.score(train_scaled, target))

0.47507051466286365


In [13]:
pred = rd.predict(test_scaled)

In [14]:
submission['box_off_num'] = pred
submission

,title,box_off_num
0,용서는 없다,1.467071e+06
1,아빠가 여자를 좋아해,4.425254e+05
2,하모니,3.675107e+06
3,의형제,2.009935e+06
4,평행 이론,1.765129e+06
...,...,...
238,해에게서 소년에게,1.547207e+05
239,울보 권투부,-3.128450e+05
240,어떤살인,4.336000e+05
241,말하지 못한 비밀,3.253510e+05


In [15]:
submission.to_csv('fourth_model.csv', index=False) # 점수 : 3835274.355285752